In [1]:
#import dependencies 
import requests
import json
import matplotlib as plt
from pprint import pprint
import pandas as pd

In [4]:
# pulling Csv file
police = "Resources/police_data.csv"
police_df = pd.read_csv(police, encoding="ISO-8859-1")

#police_df.head()
#police_df.count()

In [8]:
#Cleaning Data  
police_df = police_df.dropna(how='any')


#police_df.count()

Incident Number w/year     91997
Year of Incident           91997
Watch                      91997
Call (911) Problem         91997
Type of Incident           91997
Type  Location             91997
Date1 of Occurrence        91997
Month1 of Occurence        91997
Day1 of the Week           91997
Time1 of Occurrence        91997
Call Dispatch Date Time    91997
Victim Type                91997
Modus Operandi (MO)        91997
Update Date                91997
X Coordinate               91997
Y Cordinate                91997
Zip Code                   91997
City                       91997
State                      91997
Location1                  91997
dtype: int64

#we went with the complete data set in order to consitent an clean infromation accross the board 